# 일급 함수 디자인 패턴

## 전략 패턴의 리팩터링
전략 패턴은 파이썬에서 함수를 일급 객체로 사용하면 더욱 간단해질 수 있는 디자인 패턴의 대표적인 사례이다.

디자인 패턴에서는 전략 패턴을 다음과 같이 설명한다.
> 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에서 넣어서 교체하기 쉽게 만든다.
> 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다.

In [17]:
import inspect
# 플러그형 할인 전략을 가진 Order 클래스 구현
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order: # Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC): # 전략 추상 베이스 클래스

    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환한다."""

class FidelityPromo(Promotion): # 첫 번째 구체적인 전략
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): # 두 번째 구체적인 전략
    """20개 이상의 동일 상품을 구매하면 10% 할인 적용"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion): # 세 번째 구체적인 전략
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [18]:
# 여러 프로모션 할인을 적용해서 Order 클래스를 사용하는 예
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)
        ]

print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, BulkItemPromo()))

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, LargeOrderPromo()))
print(Order(joe, cart, LargeOrderPromo()))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


## 함수지향 전략
위 예제에서 각각의 구체적인 전략은 `discount()`라는 메서드 하나를 가진 클래스다. 게다가 전략 객체는 상태(객체 속성)를 가지고 있지 않다.
구체적인 전략 객체가 일반 함수로 보인다고 생각하면 제대로 본 것이다.
다음 예제는 전략을 간단히 함수로 변경했고 Promotion 추상 클래스를 제거했다.

In [21]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order: # Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구매하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [22]:
# 할인 전략을 함수로 정의한 Order 클래스의 사용 예
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)
        ]

print(Order(joe, cart, fidelity_promo))
print(Order(ann, cart, fidelity_promo))

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, bulk_item_promo))

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, large_order_promo))
print(Order(joe, cart, large_order_promo))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


## 최선의 전략 선택하기: 단순한 접근법

아래 예제는 가독성이 좋고 제대로 작동하지만, 일부 코드가 중복되어 있어 버그가 생길 여지가 있다.
새로운 할인 전략을 추가하려면 함수를 코딩하고 이 함수를 promos 리스트에 추가해야 한다.
아니면 새로운 할인 함수를 Order 객체에 인수로 전달해서 작동시킬 수 있지만, 이때 `best_promo()`는 새로운 할인 함수를 고려하지 않는다.

In [ ]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_proto(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

### 모듈에서 전략 찾기
파이썬 모듈도 일급 객체로서, 모듈을 다루는 여러 함수가 표준 라이브러리에서 제공된다. 파이썬 문서에서는 `globals()` 내장 함수를 다음과 같이 설명하고 있다.

> 현재 전역 심벌 테이블을 나타내는 딕셔너리 객체를 반환한다. 이 딕셔너리는 언제나 현재 모듈에 대한 내용을 담고 있다
> (함수나 메서드 안에서 호출할 때, 함수를 호출한 모듈이 아니라 함수가 정의된 모듈을 나타낸다.)

In [ ]:
# 모듈 전역 네임스페이스를 내부 조사해서 만든 promos 리스트
promos = [globals()[name] for name in globals()
          if name.endwith('_promo')
          and name != 'best_promo']

def best_promo(order):
    """최대로 할인받을 금액을 반환한다.
    """
    return max(promo(order) for promo in promos)

In [ ]:
# 새로운 promotions 모듈을 내부 조사해서 만든 promos 리스트
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_proto(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)